In [11]:
#fist exercise
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

In [49]:
df = pd.read_excel("CSM_dataset.xlsx")
df.head

<bound method NDFrame.head of                       Movie  Year  Ratings  Genre      Gross       Budget  \
0                   13 Sins  2014      6.3      8       9130    4000000.0   
1            22 Jump Street  2014      7.1      1  192000000   50000000.0   
2            3 Days to Kill  2014      6.2      1   30700000   28000000.0   
3    300: Rise of an Empire  2014      6.3      1  106000000  110000000.0   
4         A Haunted House 2  2014      4.7      8   17300000    3500000.0   
..                      ...   ...      ...    ...        ...          ...   
226                Child 44  2015      6.4      4    1210000   50000000.0   
227                   Aloha  2015      5.5     15   21000000   37000000.0   
228     Unfinished Business  2015      5.4      8   10200000   35000000.0   
229                War Room  2015      5.4      1   12300000    3000000.0   
230             The Gallows  2015      4.4     15   22600000     100000.0   

     Screens  Sequel  Sentiment    Views  Lik

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Movie                231 non-null    object 
 1   Year                 231 non-null    int64  
 2   Ratings              231 non-null    float64
 3   Genre                231 non-null    int64  
 4   Gross                231 non-null    int64  
 5   Budget               230 non-null    float64
 6   Screens              221 non-null    float64
 7   Sequel               231 non-null    int64  
 8   Sentiment            231 non-null    int64  
 9   Views                231 non-null    int64  
 10  Likes                231 non-null    int64  
 11  Dislikes             231 non-null    int64  
 12  Comments             231 non-null    int64  
 13  Aggregate Followers  196 non-null    float64
dtypes: float64(4), int64(9), object(1)
memory usage: 25.4+ KB


In [5]:
df.describe()

,Year,Ratings,Genre,Gross,Budget,Screens,Sequel,Sentiment,Views,Likes,Dislikes,Comments,Aggregate Followers
count,231.000000,231.000000,231.000000,2.310000e+02,2.300000e+02,221.000000,231.000000,231.000000,2.310000e+02,231.000000,231.000000,231.000000,1.960000e+02
mean,2014.294372,6.441558,5.359307,6.806603e+07,4.792173e+07,2209.244344,1.359307,2.809524,3.712851e+06,12732.536797,679.051948,1825.701299,3.038193e+06
std,0.456750,0.988765,4.141611,8.890289e+07,5.428825e+07,1463.767755,0.967241,6.996775,4.511104e+06,28825.484481,1243.929481,3571.040447,4.886278e+06
min,2014.000000,3.100000,1.000000,2.470000e+03,7.000000e+04,2.000000,1.000000,-38.000000,6.980000e+02,1.000000,0.000000,0.000000,1.066000e+03
25%,2014.000000,5.800000,1.000000,1.030000e+07,9.000000e+06,449.000000,1.000000,0.000000,6.233020e+05,1776.500000,105.500000,248.500000,1.830250e+05
50%,2014.000000,6.500000,3.000000,3.740000e+07,2.800000e+07,2777.000000,1.000000,0.000000,2.409338e+06,6096.000000,341.000000,837.000000,1.052600e+06
75%,2015.000000,7.100000,8.000000,8.935000e+07,6.500000e+07,3372.000000,1.000000,5.500000,5.217380e+06,15247.500000,697.500000,2137.000000,3.694500e+06
max,2015.000000,8.700000,15.000000,6.430000e+08,2.500000e+08,4324.000000,7.000000,29.000000,3.262678e+07,370552.000000,13960.000000,38363.000000,3.103000e+07


In [19]:
df.isnull().sum()

Movie                   0
Year                    0
Ratings                 0
Genre                   0
Gross                   0
Budget                  1
Screens                10
Sequel                  0
Sentiment               0
Views                   0
Likes                   0
Dislikes                0
Comments                0
Aggregate Followers    35
dtype: int64

In [41]:
df['Budget'].fillna(df['Budget'].mean(), inplace=True)
df['Screens'].fillna(df['Screens'].mean(), inplace=True )
df.isnull().sum()

Movie                   0
Year                    0
Ratings                 0
Genre                   0
Gross                   0
Budget                  0
Screens                 0
Sequel                  0
Sentiment               0
Views                   0
Likes                   0
Dislikes                0
Comments                0
Aggregate Followers    35
dtype: int64

In [42]:
#checking for duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [50]:
X = df.drop(columns=['Gross','Movie'])
y = df['Gross']


In [60]:
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

num_imputer = SimpleImputer(strategy='median')
X_num = num_imputer.fit_transform(X[num_cols])

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

if cat_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_cat_raw = cat_imputer.fit_transform(X[cat_cols])

    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
    X_cat = ohe.fit_transform(X_cat_raw)

    X_processed = np.hstack([X_num_scaled, X_cat])
else:
    X_processed = X_num_scaled


if np.isnan(X_processed).any():
    print("⚠️ هنوز NaN داریم، حذف می‌کنیم...")
    # حذف سطرهایی که NaN دارند
    mask = ~np.isnan(X_processed).any(axis=1)
    X_processed = X_processed[mask]
    y = y[mask]


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)   


In [62]:
model = SGDRegressor(
    loss='squared_error',
    penalty='l2',
    max_iter=1000,
    tol=1e-3,
    random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

Mean Squared Error: 2102521855128171.50
